In [1]:
from flask import Flask
from flask_restful import Resource, Api, reqparse, abort, fields, marshal_with
from flask_sqlalchemy import SQLAlchemy

In [2]:
app = Flask(__name__)
api = Api(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///sqlite.db'
db = SQLAlchemy(app)

G:\anaconda\lib\site-packages\flask_sqlalchemy\__init__.py:873: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
class AudioFileStimulator(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    song = db.Column(db.String(40))
    singer = db.Column(db.String(20))    

In [4]:
db.create_all()

In [5]:
song_post_args = reqparse.RequestParser()
song_post_args.add_argument("song", type=str, help='song is required', required=True) 
song_post_args.add_argument("singer", type=str, help='singer is required', required=True)

In [6]:
song_put_args = reqparse.RequestParser()
song_put_args.add_argument("song", type=str) 
song_put_args.add_argument("singer", type=str)

In [7]:
resource_fields = {
    'id' : fields.Integer,
    'song' : fields.String,
    'singer' : fields.String,
}

In [8]:
class AudioStimulator(Resource):
    def get(self):
        songs = AudioFileStimulator.query.all()
        song_list = {}
        for song in songs:
            song_list[song.id] = {'song': song.song, 'singer': song.singer}
        return song_list

In [9]:
class songlist(Resource):
    @marshal_with(resource_fields)
    def get(self, song_id):
        song = AudioFileStimulator.query.filter_by(id=song_id).first()
        if not song:
            abort(404, message='Could not find song with that ID')
        return song
    
    @marshal_with(resource_fields)
    def post(self, song_id):
        args = song_post_args.parse_args()
        song = AudioFileStimulator.query.filter_by(id=song_id).first()
        if song:
            abort(409, message='Song ID already exists')
        song_ = AudioFileStimulator(id=song_id, song=args['song'], singer=args['singer'])
        db.session.add(song_)
        db.session.commit()
        return song_, 201
    
    @marshal_with(resource_fields)
    def put(self, song_id):
        args = song_put_args.parse_args()
        song = AudioFileStimulator.query.filter_by(id=song_id).first()
        if not song:
            abort(404, message="Song_ID doesn't exist, cannot update")
        if args['song']:
            song.song = args['song']
        if args['singer']:
            song.singer = args['singer']
        db.session.commit()    
        return song
    
    def delete(self, song_id):
        song = AudioFileStimulator.query.filter_by(id=song_id).first()
        db.session.delete(song)
        return "song deleted", 204

In [10]:
api.add_resource(songlist, '/song_list/<int:song_id>')
api.add_resource(AudioStimulator, '/song_list')

In [11]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/May/2021 00:40:23] "GET http://localhost:5000/song_list HTTP/1.1" 200 -
